In [ ]:
# Collegamento a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.manifold import TSNE  # Importa TSNE
import matplotlib.pyplot as plt
import pickle
from matplotlib import cm
from skimage.color import rgb2gray

def load_images(dataset_path, target_size=(256, 256)):
    """Carica tutte le immagini dal dataset, rispettando la struttura del dataset descritto nello script di divisione."""
    images = []
    labels = []  # Per tracciare provenienza (centro o sequenza)

    # Esplora le cartelle principali (centri e sequenze)
    for main_dir in os.listdir(dataset_path):
        main_path = os.path.join(dataset_path, main_dir)

        # Se si tratta di un centro (data_Cx)
        if main_dir.startswith("data_C") and os.path.isdir(main_path):
            center_id = main_dir[-1]  # Estrai l'ID del centro (1-6)
            image_dir = os.path.join(main_path, f"images_C{center_id}")
            if os.path.exists(image_dir):
                for img_name in os.listdir(image_dir):
                    img_path = os.path.join(image_dir, img_name)
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        try:
                            img = Image.open(img_path)
                            img = img.resize(target_size)  # Ridimensiona l'immagine
                            images.append(np.array(img).flatten())
                            labels.append(main_dir)
                        except Exception as e:
                            print(f"Errore nel caricamento immagine {img_path}: {e}")

         # Se si tratta di una sequenza (seqX)
        elif main_dir.startswith("sequenceData") and os.path.isdir(main_path):
            image_dir1 = os.listdir(main_path)
            for a in image_dir1:
              image_dir = os.path.join(main_path, a, f"images_{a}")
              if os.path.exists(image_dir):
                  for img_name in os.listdir(image_dir):
                      img_path = os.path.join(image_dir, img_name)
                      if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                          try:
                              img = Image.open(img_path)
                              img = img.resize(target_size)  # Ridimensiona l'immagine
                              images.append(np.array(img).flatten())  # Appiattisci l'immagine
                              labels.append(main_dir)
                          except Exception as e:
                              print(f"Errore nel caricamento immagine {img_path}: {e}")

    return np.array(images), labels


# Percorsi principali
dataset_path = "/content/drive/MyDrive/CHALLENGE/PolypGen-EIM-24-25"
output_tsne_path = "/content/drive/MyDrive/CHALLENGE/tsne_plane.pkl"  # Salva il piano TSNE

# Caricamento immagini
print("Caricamento immagini dal dataset...")
images, labels = load_images(dataset_path)
print(f"Numero totale di immagini caricate: {len(images)}")



In [ ]:
# T-SNE
print("Calcolo T-SNE...")
tsne = TSNE(n_components=2, random_state=42)  # Aggiungi il parametro random_state per risultati ripetibili
tsne_result = tsne.fit_transform(images)

In [ ]:
# Assegna colori alle etichette
unique_labels = list(set(labels))  # Trova etichette uniche (centri e sequenze)
label_to_color = {label: i for i, label in enumerate(unique_labels)}  # Mappa etichetta -> indice
colors = [label_to_color[label] for label in labels]  # Converte etichette in indici numerici

# Usare una mappa colori
colormap = cm.get_cmap('tab10', len(unique_labels))  # Usa una mappa con 10 colori (o più se necessario)
color_values = [colormap(c) for c in colors]  # Ottieni colori per ogni punto

# Salvataggio del piano TSNE
with open(output_tsne_path, 'wb') as f:
    pickle.dump(tsne, f)
print(f"Piano TSNE salvato in: {output_tsne_path}")

# Plot dei risultati
plt.figure(figsize=(12, 10))
scatter = plt.scatter(tsne_result[:, 0], tsne_result[:, 1], alpha=0.7, c=color_values, edgecolor='k')
plt.title('T-SNE delle immagini del dataset (RGB)')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')

# Aggiungi legenda
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=colormap(i), markersize=10) for i in range(len(unique_labels))]
plt.legend(handles, unique_labels, title="Centri e Sequenze", loc="best")

plt.grid(True)
plt.savefig("tsne_plot_colored.png")  # Salva il grafico
plt.show()

print("Elaborazione completata con successo.")


In [ ]:
# Varianza dopo il T-SNE
tsne_variance = np.var(tsne_result, axis=0).sum()

print(f"Varianza dopo il T-SNE: {tsne_variance}")
